In [1]:
# Import library-library
import os
import glob

# Data Preparation and Preprocessing
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
NLTK_StopWords = stopwords.words('indonesian')
NLTK_StopWords = set(NLTK_StopWords)

# Word Embedding
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

from sklearn.metrics.pairwise import cosine_similarity

# Input and Expansion Query
from textblob import TextBlob
import nltk
from nltk.tokenize import wordpunct_tokenize
#from googletrans import Translator

#Split to train and test
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
excluded_words = ["tempat", "waktu", "hari"]

NLTK_StopWords = stopwords.words('indonesian')
NLTK_StopWords.extend(["detik", "detikjatim", "detikjateng", "detikjabar", "detiksulsel", "detiksumbar", "detikbali", "detikpapua", "detiksulteng", "detikmaluku", "detjatim", "detikcom", "allahumma", "aamiin", "allah", "bismillah"])
NLTK_StopWords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])
# txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
# NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
NLTK_StopWords = set(NLTK_StopWords)

In [3]:
def preprocessing(berita):
    s = str(berita)
    s = s.lower()
    s = s.replace('\n', ' ')
    s = s.replace('\r', ' ')
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]
    T = [t for t in tokens if ((t in excluded_words) or (t not in NLTK_StopWords))]
    return T

def preprocessing2(berita):
    s = str(berita)
    s = s.lower()
    s = s.replace('\n', ' ')
    s = s.replace('\r', ' ')
    tokens = [token for token in s.split(" ") if token != ""]
    return tokens

In [4]:
text=preprocessing('saya Mencoba menggunakan preprocessing')
print (text)

['mencoba', 'preprocessing']


In [5]:
df =pd.read_csv("/Users/adi/Desktop/TA_Adi_V2/QE Statistik V2/scrapped_news/scrapped_news.csv")
df.head()

,title,date,description,source
0,Polisi Buru Begal Penusuk Dada Pemuda Bandung!,"Rabu, 04 Mei 2022 12:26 WIB",\-\n\nPolisi bergerak menyusul adanya insiden ...,www.detik.com
1,"Kawanan Begal Beraksi di Gunung Putri Bogor, P...","Rabu, 15 Jun 2022 19:43 WIB",\-\n\nPemuda berinisial MH (19) menjadi korban...,www.detik.com
2,"Ditangkap Polisi, Ini Penampakan 9 Begal 2 Pra...","Selasa, 10 Mei 2022 14:30 WIB",\-\n\nPolres Metro Jakarta Selatan menggelar k...,www.detik.com
3,2 Anggota TNI Gagalkan 9 Begal di Kebayoran Ba...,"Senin, 09 Mei 2022 09:02 WIB",\-\n\nTNI menyatakan dua serdadunya telah meng...,www.detik.com
4,"5 Kisah Mereka yang Mengarang Cerita Dibegal, ...","Jumat, 29 Apr 2022 17:58 WIB",\-\n\nAksi kriminal pembegalan di jalan bisa m...,www.detik.com


In [6]:
column = ['Title', 'Date', 'Description','Source']
df_total = pd.DataFrame()
df =pd.read_csv("/Users/adi/Desktop/TA_Adi_V2/QE Statistik V2/scrapped_news/scrapped_news.csv")
# print(df.head())
isi=df['description']
df = df.dropna(subset=['description'], axis=0)
text = preprocessing (df.iloc[0, -2])
text = (' '.join(text)) 

# for x in range(0, df.shape[0]):
#     text = preprocessing (df.iloc[x, -2])
#     # document_text.append(df.iloc[x, -2])
#     df.iloc[x, -2] = (' '.join(text)) 

In [7]:
df.head()
df_train,df_test=train_test_split(df,test_size=0.1)
df_train.head()

,title,date,description,source
52,Maling Motor Parlente di Pasuruan Gondol Motor...,"Selasa, 15 Mar 2022 03:06 WIB",\-\n\nMaling perlente beraksi di sebuah klinik...,www.detik.com
1239,2 Komplotan Maling Motor Asal Tangerang dan Le...,"Selasa, 26 Apr 2022 16:55 WIB",\-\n\nPolsek Neglasari meringkus dua komplotan...,www.detik.com
651,Pria di Dairi Nekat Habisi Oppungnya demi Perh...,"Kamis, 26 Mei 2022 08:29 WIB","\-\n\nSeorang pria di Kabupaten Dairi, bernama...",www.detik.com
100,"Korban Curanmor Silakan Merapat, Ada 7 Mobil-4...","Kamis, 03 Mar 2022 01:31 WIB",\-\n\nPolres Bogor menyita 7 mobil dan 48 moto...,www.detik.com
1115,"Kepergok Curi Motor Mahasiswa USU, 2 Pria Ini ...","Senin, 06 Jun 2022 14:35 WIB",\-\n\n2 pelaku pencuri sepeda motor milik maha...,www.detik.com


In [9]:
df_train_noPre=df_train
df_train_noPre.to_csv('corpus/dataset/df_train_noPre.csv', index=False)
df_test_noPre=df_test
df_test_noPre.to_csv('corpus/dataset/df_test_noPre.csv', index=False)

In [10]:
document_text_train = []
document_text_test = []
for x in range(0, df_train.shape[0]):
    text = preprocessing(df_train.iloc[x,-2])
    document_text_train.append(df_train.iloc[x, -2])
    df_train.iloc[x,-2] = (' '.join(text))
for x in range(0, df_test.shape[0]):
    text = preprocessing(df_test.iloc[x,-2])
    document_text_test.append(df_test.iloc[x, -2])
    df_test.iloc[x,-2] = (' '.join(text))

/Users/adi/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/adi/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [11]:
teks_train = []
teks_test = []

for i in range(df_test.shape[0]):
        teks_test.append(df_test.iloc[i, 2])
for i in range(df_train.shape[0]):
        teks_train.append(df_train.iloc[i, 2])

In [12]:
print(df_test.info())
print(df_train.info())
print ('-'*90)
print("Panjang Dokumen Teks : ", len(document_text_test))
print("Panjang Dokumen Teks : ", len(document_text_train))
#print(len(document_text))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135 entries, 19 to 130
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        135 non-null    object
 1   date         135 non-null    object
 2   description  135 non-null    object
 3   source       135 non-null    object
dtypes: object(4)
memory usage: 5.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1207 entries, 52 to 362
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1207 non-null   object
 1   date         1207 non-null   object
 2   description  1207 non-null   object
 3   source       1207 non-null   object
dtypes: object(4)
memory usage: 47.1+ KB
None
------------------------------------------------------------------------------------------
Panjang Dokumen Teks :  135
Panjang Dokumen Teks :  1207


In [13]:
# Simpan teks asli dokumen
joblib.dump(document_text_test, "corpus/model/document_text_test.pkl")
joblib.dump(document_text_train, "corpus/model/document_text_train.pkl")
# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
df.to_csv('corpus/dataset/df_total.csv',index=False )
df_train.to_csv('corpus/dataset/df_train.csv', index=False)
df_test.to_csv('corpus/dataset/df_test.csv', index=False)


In [14]:
# Fit transform menggunakan TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_test_matrix = tfidf_vectorizer.fit_transform(teks_test)
tfidf_train_matrix = tfidf_vectorizer.fit_transform(teks_train)

print('tfidf_test_matrix')
print(tfidf_test_matrix.shape)
print('tfidf_train_matrix')
print(tfidf_train_matrix.shape)
# print(tfidf_matrix)

tfidf_test_matrix
(135, 4423)
tfidf_train_matrix
(1207, 14075)


In [15]:
joblib.dump(tfidf_vectorizer, "corpus/vectorizer/vectorizer.pkl")
joblib.dump(tfidf_test_matrix, "corpus/matrix/tfidf_test.pkl")
joblib.dump(tfidf_train_matrix, "corpus/matrix/tfidf_train.pkl")

['corpus/matrix/tfidf_train.pkl']

In [16]:
df_train2=pd.read_csv('corpus/dataset/df_train_noPre.csv')
document_text_train2=[]
for x in range(0, df_train2.shape[0]):
    text = preprocessing2(df_train2.iloc[x,-2])
    document_text_train2.append(df_train2.iloc[x, -2])
    df_train2.iloc[x,-2] = (' '.join(text))


df_train2.to_csv('corpus/dataset/df_train_lower.csv', index=False)

In [17]:
df_test2=pd.read_csv('corpus/dataset/df_test_noPre.csv')
document_text_test2=[]
for x in range(0, df_test2.shape[0]):
    text = preprocessing2(df_test2.iloc[x,-2])
    document_text_test2.append(df_test2.iloc[x, -2])
    df_test2.iloc[x,-2] = (' '.join(text))


df_test2.to_csv('corpus/dataset/df_test_lower.csv', index=False)

### Pencarian 

In [16]:
df_total = pd.read_csv('df_total.csv')
df_total = df_total[pd.notnull(df_total['description'])]
df_test = pd.read_csv('df_test.csv')
df_test = df_test[pd.notnull(df_test['description'])]
print(df_total.info())
print ('-'*90)
print(df_test.info())
print ('-'*90)
document_text_train= joblib.load('document_text_train.pkl')
document_text_test= joblib.load('document_text_test.pkl')
print(len(document_text_test))
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2320 entries, 0 to 2319
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        2320 non-null   object
 1   date         2320 non-null   object
 2   description  2320 non-null   object
 3   source       2320 non-null   object
dtypes: object(4)
memory usage: 90.6+ KB
None
------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 232 entries, 0 to 231
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        232 non-null    object
 1   date         232 non-null    object
 2   description  232 non-null    object
 3   source       232 non-null    object
dtypes: object(4)
memory usage: 9.1+ KB
None
------------------------------------------------------------------------------------------
232
2088


In [17]:
kueri='Lalin truk di jakarta'
print (kueri)
kueri=preprocessing(kueri)
kueri= [" ".join (kueri)]
print (kueri)

tfidf_matrix =joblib.load( "tfidf_train.pkl" )
tfidf_vectorizer = joblib.load( "vectorizer.pkl" ) 
query_vec= tfidf_vectorizer.transform(kueri)
results=cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
#print (results)
print("\n======================\n")
print("Top 10 most similar documents in corpus:")

j = 1

for i in results.argsort()[-10:][::-1]:
    print(j)
    print("No ID Dokumen  : ", i)
    print("Tanggal        : ", df_total.iloc[i,1])
    print("Isi berita     : ", df_total.iloc[i,2])
    print("(Score: %.4f) " % results[i])
    j += 1

Lalin truk di jakarta
['lalin truk jakarta']


Top 10 most similar documents in corpus:
1
No ID Dokumen  :  941
Tanggal        :  Senin, 09 Agu 2021 13:30 WIB
Isi berita     :  Satu unit truk bermuatan batu lepas kontrol dan menabrak pemotor dan pejalan
kaki di Kulon Progo, Daerah Istimewa Yogyakarta (DIY) hari ini. Akibatnya dua
orang tewas dan dua orang lainnya mengalami luka-luka.

 Peristiwa itu bermula saat truk melaju di jalanan menurun di wilayah Kalurahan
Hargorejo, Kapanewon Kokap, Kulon Progo, sekitar pukul 09.00 WIB tadi.
Setibanya di belokan dekat jembatan, truk itu lepas kontrol dan menabrak
pengendara sepeda motor, serta seorang pejalan kaki hingga akhirnya meluncur
ke jurang.

 "Berdasarkan informasi yang kami himpun, diduga pengemudi truk kurang
konsentrasi, lalu menyerempet pengendara motor yang dinaiki dua orang pelajar,
kemudian menabrak pejalan kaki, hingga akhirnya truk itu jatuh ke dalam
jurang," ungkap penyidik Unit Laka Lantas Satlantas Polres Kulon Progo, Bripk